In [3]:
# !pip install python==3.9
!pip install torch
!pip install deepparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 5.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 48.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 66.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29.76:
      Successfully uninstalled botocore-1.29.76
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
# import pytorch
from deepparse.parser import AddressParser
import deepparse


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
def generate_rand_garbage(list_bloc):
    list_garbage = ['',' ','École','Bibliothèque','Musée','Bureau de poste','Banque','Hôtel','Restaurant','Café','Cinéma','Théâtre',
                    'Stade','Gymnase','Parc aquatique','Magasin de vêtements','Pharmacie','Épicerie','Salon de coiffure','Station service',
                    'Garage','Boutique de jouets','Galerie d art','Studio de danse','Salle de concert','Centre commercial','Boulangerie',
                    'Université','Poste de police','Caserne de pompiers','Église','Mosquée','Synagogue','Temple','Hôpital','Clinique','Cabinet médical',
                    'Pharmacie','Maison de retraite','Centre de réadaptation','Salle de conférence','Salle d exposition']
    garbage = random.choice(list_garbage)
    list_extension = ['', ' de ' + list_bloc[4], ' van ' + list_bloc[4]]
    extension = random.choice(list_extension)
    if garbage == '': 
        return ''
    else : 
        return garbage + extension

def del_symbols(list_bloc):
    result = []
    symbols = "!@#$%^&*()_-+={}[]|\:;'<>?,./\""
    for item in list_bloc:
        new_item = ''.join([c if c.isalnum() or c.isspace() else ' ' for c in item])
        result.append(new_item)
    return result

def generate_rand_presence(list_bloc):
    N = len(list_bloc)
    random_list = [random.choices([0,1], [0.2,0.8])[0] for _ in range(N)]
    return random_list

def generate_rand_order(list_bloc):
    N = len(list_bloc)
    unique_list = list(range(1, N + 1))
    random.shuffle(unique_list)
    return unique_list

def add_typo_error(list_bloc, list_order):
    result = []
    error_type = random.choice(['nothing','add', 'delete'])
    for i in range(len(list_bloc)):
        string = list_bloc[i]
        order = list_order[i]
        if error_type == 'nothing' or order=='PostalCode': 
            pass
        elif error_type == 'add':
            error_index = random.randint(0, len(string))
            error_char = random.choice('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
            string = string[:error_index] + error_char + string[error_index:]
        else:
            if len(string) > 0:
                error_index = random.randint(0, len(string) - 1)
                string = string[:error_index] + string[error_index+1:]
        result.append(string)
    return result

def generate_elem_category(list_bloc, list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery']):
    list_elem_category = []
    if len(list_bloc) != len(list_order):
        print('il n\'y a pas autant de blocs que de categories ' + str(len(list_bloc)) + ' != ' + str(len(list_order)))
        raise 'il n\'y a pas autant de blocs que de categories ' + str(len(list_bloc)) + ' != ' + str(len(list_order))
    for i in range(len(list_bloc)):
        bloc = list_bloc[i]
        order = list_order[i]
        len_bloc = len(bloc.split())
        elem_category = [order] * len_bloc
        list_elem_category.append(elem_category)
    return list_elem_category

def change_list_order(lst, order):
    if len(lst) != len(order):
        raise ValueError("List and order must have the same length.")
    new_lst = [None] * len(lst)
    for i, value in enumerate(lst):
        new_index = order[i] - 1  
        new_lst[new_index] = value
    return new_lst

def address_augmentation(list_bloc, list_order):
    
    list_bloc[-1] = generate_rand_garbage(list_bloc)
    # print(list_bloc)
    list_bloc = del_symbols(list_bloc)
    # print(list_bloc)
    list_bloc = add_typo_error(list_bloc, list_order)
    # print(list_bloc)
    list_elem_cat = generate_elem_category(list_bloc)
    rand_presence = generate_rand_presence(list_bloc)
    rand_order = generate_rand_order(list_bloc)
    
    list_bloc_ordered = change_list_order(list_bloc,rand_order)
    list_elem_cat_ordered = change_list_order(list_elem_cat,rand_order)
    rand_presence_ordered = change_list_order(rand_presence,rand_order)
    
    list_final_category = []
    string_final = ''
    for i in range(len(rand_presence_ordered)):
        if rand_presence_ordered[i] == 1:
            list_final_category.append(list_elem_cat_ordered[i])
            if i == 0:
                string_final = list_bloc_ordered[0]
            else :
                string_final = string_final + ' ' + list_bloc_ordered[i]
    string_final = string_final
    flattened_list_final_category = [element for sublist in list_final_category for element in sublist]
    
    return string_final, flattened_list_final_category

In [ ]:
!pip install pytest
!pip install mock

In [ ]:
from unittest import mock

# ['StreetName','StreetNumber','Unit','PostalCode','Municipality','GeneralDelivery']
list_bloc1 = ['86','Avenue des Camélias','','1150','Woluwé-Saint-Pierre','']
list_bloc2 = ['22','Belgielaan','','1800','Vilvoorde','']
list_bloc3 = ['115','Rue inventée','4','1200','Woluwé-Saint-Lambert','Boutique de jouets']
list_bloc4 = ['8','Avenue de Boileau','5','1160','Auderghem','Magasin de vêtements de Auderghem']
list_bloc5 = ['8','Avenue de Boileau','5','1160','Auderghem','']
list_bloc6 = ['115','Rue inventée','4','1200','Woluwé-Saint-Lambert','']

# test part
def test_generate_rand_garbage():
    with mock.patch('random.choice') as mock_choice:
        mock_choice.side_effect = ['',' de ' + list_bloc1[4]]
        g = generate_rand_garbage(list_bloc1)
        assert g == '', g
    with mock.patch('random.choice') as mock_choice:
        mock_choice.side_effect = ['Bibliothèque', ' de ' + list_bloc2[4]]
        g = generate_rand_garbage(list_bloc2)
        assert g == 'Bibliothèque de Vilvoorde', g

def test_del_symbols():
    g1 = del_symbols(list_bloc1)
    assert g1 == ['86','Avenue des Camélias','','1150','Woluwé Saint Pierre',''], g1
    g2 = del_symbols(list_bloc6)
    assert g2 == ['115','Rue inventée','4','1200','Woluwé Saint Lambert',''], g2

def test_generate_rand_presence():
    g1 = generate_rand_presence(list_bloc1)
    assert sum(g1) <= 6, g1
    g2 = generate_rand_presence(list_bloc2)
    assert sum(g2) >= 0, g2

def test_generate_rand_order():
    g = generate_rand_order(list_bloc3)
    g_sorted = sorted(g)
    assert g_sorted == [1,2,3,4,5,6], g

'''
def test_add_typo_error():
    with mock.patch('random.randint') as mock_index, \
         mock.patch('random.choice') as mock_choice:  
        mock_index.return_value = 5
        mock_choice.return_value = 'delete'
        assert add_typo_error(list_bloc) == 
'''

# ['8','Avenue de Boileau','5','1160','Auderghem','Magasin de vêtements de Auderghem']
def test_generate_elem_category():
    g = generate_elem_category(list_bloc4, list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery'])
    assert g == \
        [['StreetNumber'],['StreetName','StreetName','StreetName'],['Unit'],['PostalCode'],['Municipality'],['GeneralDelivery','GeneralDelivery','GeneralDelivery','GeneralDelivery','GeneralDelivery']], g

"""
def test_change_list_order():
    assert change_list_order(lst, order) == 

def test_address_augmentation():
    assert address_augmentation(list_bloc) == 
"""

test_generate_rand_garbage()
test_del_symbols()
test_generate_rand_presence()
test_generate_rand_order()
test_generate_elem_category()

In [ ]:
list_add = [list_bloc1,list_bloc2,list_bloc3,list_bloc4,list_bloc5,list_bloc6]
list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery']
print(list_add)
for s in list_add:    
    # print(s)
    # print(type(s))
    # print(len(s))
    print(address_augmentation(s,list_order))

Dataset Preprocessing part 1: join muni and best

In [25]:
df_muni = pd.read_csv('/kaggle/input/municipalities/muni.csv')
df_muni.columns = 'muni_' + df_muni.columns
df_muni['muni_validity_from'] = pd.to_datetime(df_muni['muni_validity_from'])
df_muni = df_muni.sort_values('muni_validity_from', ascending=False)
df_muni = df_muni.drop_duplicates(subset='muni_muni_code',keep='first').reset_index()
print(df_muni)

# df_muni = df_muni[['muni_muni_code','muni_name_fr','muni_name_nl', 'muni_name_de']]

df_muni_fr= df_muni[['muni_muni_code','muni_name_fr']]
df_muni_fr.columns = ['muni_muni_code','muni_name']

df_muni_nl= df_muni[['muni_muni_code','muni_name_nl']]
df_muni_nl.columns = ['muni_muni_code','muni_name']

df_muni_de= df_muni[['muni_muni_code','muni_name_de']]
df_muni_de.columns = ['muni_muni_code','muni_name']

del df_muni
df_muni = df_muni_fr
del df_muni_fr
df_muni = pd.concat([df_muni, df_muni_nl], axis=0)
del df_muni_nl
df_muni = pd.concat([df_muni, df_muni_de], axis=0)
del df_muni_de

print(df_muni)

     index  muni_sid  muni_muni_code       muni_name_fr       muni_name_nl  \
0     3501      3501           93090          Viroinval          Viroinval   
1     3138      3113           32010          Koekelare          Koekelare   
2     3118      3094           25118           Hélécine           Hélécine   
3     3117      3093           25117            Chastre            Chastre   
4     3116      3092           25112              Wavre              Waver   
..     ...       ...             ...                ...                ...   
602     24      1081           72040  Meeuwen-Gruitrode  Meeuwen-Gruitrode   
603     19      1076           72029           Overpelt           Overpelt   
604   1037       628           12030              Puurs              Puurs   
605   1038       629           12034        Sint-Amands        Sint-Amands   
606   1560      1063           71047         Opglabbeek         Opglabbeek   

          muni_name_de muni_validity_from muni_validity_to  \
0

In [22]:
df_best = pd.read_csv('/kaggle/input/best-address/best.csv')
df_best.columns = 'best_' + df_best.columns

df_best = df_best[['best_streetname_fr', 'best_streetname_nl',
                  'best_streetname_de', 'best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']]

df_best

# container_train = deepparse.dataset_container.ListDatasetContainer(list_train)

/tmp/ipykernel_33/4031768811.py:1: DtypeWarning: Columns (2,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_best = pd.read_csv('/kaggle/input/best-address/best.csv')


,best_streetname_fr,best_streetname_nl,best_streetname_de,best_house_number,best_box_number,best_muni_code,best_poco_code
0,Rue Rasson,Rassonstraat,NaN,87,NaN,21015,1030
1,Avenue Maréchal Ney,Maarschalk Neylaan,NaN,55,NaN,21016,1180
2,Avenue du Centaure,Centauruslaan,NaN,2,NaN,21018,1200
3,Avenue Henri Hollevoet,Henri Hollevoetlaan,NaN,14,NaN,21012,1080
4,Rue Vandeweyer,Vandeweyerstraat,NaN,43,NaN,21015,1030
...,...,...,...,...,...,...,...
6556525,Grand-Rue,NaN,NaN,223,NaN,85009,6740
6556526,Rue Sauvenière,NaN,NaN,3,NaN,61048,4590
6556527,Rue Sauvenière,NaN,NaN,6,NaN,61048,4590
6556528,Rue Sauvenière,NaN,NaN,1A,NaN,61048,4590


In [24]:
address = pd.merge(df_best,df_muni,left_on=['best_muni_code'],right_on=['muni_muni_code'])
print(address)

address = 

                 best_streetname_fr          best_streetname_nl  \
0                        Rue Rasson                Rassonstraat   
1                    Rue Vandeweyer            Vandeweyerstraat   
2                Rue Général Gratry       Generaal Gratrystraat   
3        Place de Jamblinne de Meux   de Jamblinne de Meuxplein   
4          Rue Alexandre Markelbach  Alexandre Markelbachstraat   
...                             ...                         ...   
6556525                 Rue de Redu                         NaN   
6556526               Rue de Wellin                         NaN   
6556527             Rue de Sodumont                         NaN   
6556528                  Rue Laloux                         NaN   
6556529            Rue du Pont Gahy                         NaN   

        best_streetname_de best_house_number best_box_number  best_muni_code  \
0                      NaN                87             NaN           21015   
1                      NaN         

Dataset Preprocessing part 2: Augmentation

In [ ]:
address_parser = AddressParser(model_type="bpemb", device=0)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session